In [13]:
import io
import pandas as pd
from pandas import json_normalize

import numpy as np
import json
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

import gspread as gs
from datetime import datetime

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [14]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [15]:
cashbac_key='Cashbac-GCP-Keys.json'
# horego_key='Horego-GCP-Keys.json'

name_exclude=get_name_exclude(cashbac_key)
subtype_exclude=get_subtype_exclude(cashbac_key)
exclude_restorant=get_exclude_restorant(cashbac_key)
include_restorant=get_include_restaurant_type1(cashbac_key)
outlet_in_mall_type1=get_include_restaurant_mall_type1(cashbac_key)

name_exclude.KEYWORD=name_exclude.KEYWORD.str.replace("'",'')
subtype_exclude.KEYWORD=subtype_exclude.KEYWORD.str.replace("'",'')

In [16]:
exclude_restorant

,place_id
2,ChIJ59dylc3xaS4R5mVTgzWVpuU
3,ChIJl1pNhyz3aS4RWpRUrzCQWAg
4,ChIJW9fjK_fnaC4RHTSEE8oc50g
5,ChIJZdRI6TbzaS4R2xBkynvBeA8
6,ChIJ_50x0I_taS4RS2Xlj3NRsLI
...,...
2232,ChIJg-Qmv68hai4Rm2FkV_yM4XA
2233,ChIJEwHBZ8j3aS4RSNyZ7Z0IDG4
2234,ChIJ_TFvu2Edai4RYNyRhKdJrek
2235,ChIJaxyHDp0Cai4RFjVyz-7GAAg


In [17]:
subtype_condition=''
for subtype in subtype_exclude.KEYWORD.unique():
    subtype_condition=subtype_condition+' '+f"""or lower(subtypes) like lower('%{subtype}%')"""

name_condition=''
for name in name_exclude.KEYWORD.unique():
    name_condition=name_condition+' '+f"""or lower(name) like lower('%{name}%')"""

exclude_restorant_condition=''
for place_id in exclude_restorant.place_id.unique():
    exclude_restorant_condition=exclude_restorant_condition+' '+f"""or lower(place_id) = lower('{place_id}')"""

In [18]:
include_restorant_condition='lower(place_id) not in ('
for place_id in include_restorant[(include_restorant.place_id!='')].place_id.unique():
    place_id=place_id.lower()
    include_restorant_condition=include_restorant_condition+' '+f"""'{place_id}',"""
include_restorant_condition=include_restorant_condition[:-1]+')'


include_restorant_condition_google_id='lower(google_id) not in ('
for google_id in include_restorant[(include_restorant.google_id!='')].google_id.unique():
    google_id=google_id.lower()
    include_restorant_condition_google_id=include_restorant_condition_google_id+' '+f"""'{place_id}',"""
include_restorant_condition_google_id=include_restorant_condition_google_id[:-1]+')'

In [19]:
include_restorant_condition2='lower(place_id) not in ('
for place_id in outlet_in_mall_type1[(outlet_in_mall_type1.place_id!='')].place_id.unique():
    place_id=place_id.lower()
    include_restorant_condition2=include_restorant_condition2+' '+f"""'{place_id}',"""
include_restorant_condition2=include_restorant_condition2[:-1]+')'


include_restorant_condition_google_id2='lower(google_id) not in ('
for google_id in outlet_in_mall_type1[(outlet_in_mall_type1.google_id!='')].google_id.unique():
    google_id=google_id.lower()
    include_restorant_condition_google_id2=include_restorant_condition_google_id2+' '+f"""'{google_id}',"""
include_restorant_condition_google_id2=include_restorant_condition_google_id2[:-1]+')'

In [55]:
# import psycopg2.pool
# def db_connect_cashbac_nopool(host,database,user,password):
#     """
#     :return:
#     """
#     connection = psycopg2.connect(host=host,
#                               database=database,
#                               user=user,
#                               password=password,
#                               port=5432)
#     return connection

In [56]:
# try:
#     cursor.close()
#     conn_details.close()
# except:
#     pass
    
# conn_details=db_connect_cashbac_nopool('uat-cashbac-db.ciwxizsaix0m.ap-southeast-1.rds.amazonaws.com','cashbacdb','data_ds','data123')
# cursor = conn_details.cursor()

# for subtype in tqdm(subtype_exclude.KEYWORD):

#     delete_query=f"""
#         DELETE FROM cbmerchant.outlet
#         WHERE true
#         and lower(outlet_tags_en::text) like lower('%{subtype}%')
#     """

#     cursor.execute(delete_query)
#     conn_details.commit()
    
# cursor.close()
# conn_details.close()

In [20]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

# credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
# project_id = 'horego-big-query'

# # Make clients.
# horego_bqclient = bigquery.Client(
#     credentials=credentials,
#     project=project_id,
# )

In [12]:
%%time
#######################################################################################

query_string=f""" 

select * from 
(
    SELECT * 
    FROM 
    (
        select * from cashbac-31433.cashbac_datalake_prod.outcrapper_google_results 
        union all 
        SELECT * FROM cashbac-31433.cashbac_datalake_prod.outcrapper_google_results_mall
    )table1
    where 
    (
        {subtype_condition[3:]}
    )
    or 
    (
        {name_condition[3:]}
    )
    or 
    (
        {exclude_restorant_condition[3:]}
    )
)table1
where true
and lower(category) not like '%cafe%'
and lower(category) not like '%coffee shops%'
and lower(category) not like '%restaurant%' 
and lower(category) not like '%restoran%' 
and lower(category) not like '%bars%'
and not exists
(
    select 1 
    FROM `cashbac-31433.cashbac_datalake_prod.list_delete_in_prod` list1
    where true 
    and list1.place_id = table1.place_id
    and inactive_places_date is null

)
and 
(
    {include_restorant_condition}
)
and 
(
    {include_restorant_condition_google_id}
)
and 
(
    {include_restorant_condition2}
)
and 
(
    {include_restorant_condition_google_id2}
)
"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))
    

del query_result

0
CPU times: user 99.8 ms, sys: 34 ms, total: 134 ms
Wall time: 5.27 s


In [22]:
query_string=f""" 

SELECT * 
    FROM 
    (
        select * from cashbac-31433.cashbac_datalake_prod.outcrapper_google_results 
        union all 
        SELECT * FROM cashbac-31433.cashbac_datalake_prod.outcrapper_google_results_mall
    )table1
    where 
    # (
    #     {subtype_condition[3:]}
    # )
    # or 
    # (
    #     {name_condition[3:]}
    # )
    # or 
    (
        {exclude_restorant_condition[3:]}
    )
    """
query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))
    

del query_result

0


In [25]:
data

,keyword,name,site,type,subtypes,category,phone,full_address,borough,street,city,postal_code,state,us_state,country,country_code,latitude,longitude,time_zone,plus_code,area_service,rating,reviews,reviews_link,reviews_tags,reviews_per_score,reviews_per_score_1,reviews_per_score_2,reviews_per_score_3,reviews_per_score_4,reviews_per_score_5,photos_count,photo,street_view,located_in,working_hours,working_hours_old_format,other_hours,popular_times,business_status,about,range,posts,logo,description,verified,owner_id,owner_title,owner_link,reservation_links,booking_appointment_link,menu_link,order_links,location_link,place_id,google_id,cid,reviews_id,located_google_id,email_1,email_1_full_name,email_1_title,email_2,email_2_full_name,email_2_title,email_3,email_3_full_name,email_3_title,phone_1,phone_2,phone_3,facebook,instagram,linkedin,medium,reddit,skype,snapchat,telegram,whatsapp,twitter,vimeo,youtube,github,crunchbase,website_title,website_generator,website_description,website_keywords,website_has_fb_pixel,website_has_google_tag,scrappe_date,hot_area,kodepos,province,kabupaten,kecamatan,kelurahan,Mall,Floor,update_ts
0,https://www.google.com/maps/place/sopo+ngiro+j...,Sopo Ngiro Japanese Food Jakarta Pusat,None,japanese delicatessen,japanese delicatessen,toko makanan jepang,None,"Depan TPU Karet Tensil, Jl. K.H. Mas Mansyur, ...","Karet, tensil","Depan TPU Karet Tensil, Jl. K.H. Mas Mansyur",Central Jakarta City,10230,Jakarta,None,Indonesia,ID,-6.2072041,106.8170271,Asia/Jakarta,6P58QRV8+4R,False,3.5,10,None,None,None,1,2,1,3,3,7,https://lh5.googleusercontent.com/p/AF1QipOSZn...,https://streetviewpixels-pa.googleapis.com/v1/...,None,null,None,None,None,OPERATIONAL,"{""Service options"":{""Delivery"":true,""In-store ...",None,None,https://lh5.googleusercontent.com/-Dl9nlQyDbrA...,None,True,None,Sopo Ngiro Japanese Food Jakarta Pusat,https://www.google.com/maps/contrib/1119784772...,None,None,None,None,https://www.google.com/maps/place/Sopo+Ngiro+J...,ChIJ5_FuGUj3aS4R15ngShd1ZsU,0x2e69f748196ef1e7:0xc56675174ae099d7,14224185215997221335,-4222558857712330281,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2023-11-27,None,10220,DKI JAKARTA,KOTA JAKARTA PUSAT,TANAH ABANG,KARET TENGSIN,None,None,None
1,https://www.google.com/maps/place/guten+morgen...,Guten Morgen Coffee Setiabudi One,None,coffee store,coffee store,coffee store,None,Setiabudi One Mall unit B103 H.R Rasuna Said n...,"Kuningan, Karet Kuningan, Setiabudi",Setiabudi One Mall unit B103 H.R Rasuna Said n...,South Jakarta City,12940,Jakarta,None,Indonesia,ID,-6.2152394,106.8300557,Asia/Jakarta,6P58QRMJ+W2,False,4.7,3,https://search.google.com/local/reviews?placei...,None,None,0,0,0,1,2,25,https://lh5.googleusercontent.com/p/AF1QipOaAM...,https://streetviewpixels-pa.googleapis.com/v1/...,None,null,None,None,None,OPERATIONAL,"{""Service options"":{""Takeaway"":true}}",None,None,https://lh6.googleusercontent.com/-pW4TulW2yac...,None,True,None,Guten Morgen Coffee Setiabudi One,https://www.google.com/maps/contrib/1022205146...,None,None,None,None,https://www.google.com/maps/place/Guten+Morgen...,ChIJwx4k0h31aS4Rsr_8e2Q71PE,0x2e69f51dd2241ec3:0xf1d43b647bfcbfb2,17425618160966221746,-1021125912743329870,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2023-11-27,None,12940,DKI JAKARTA,KOTA JAKARTA SELATAN,SETIA BUDI,KARET KUNINGAN,None,None,None
2,https://www.google.com/maps/place/seafood+68+s...,Seafood 68 Santa,None,seafood restaurant,"seafood restaurant, crab house, padang restaurant",restaurants,None,"Petogogan, Jl. Wolter Monginsidi No.58 2 4, RT...","West Rawa, Kebayoran Baru","Petogogan, Jl. Wolter Monginsidi No.58 2 4, RT...",South Jakarta City,12170,Jakarta,None,Indonesia,ID,-6.2391552,106.8112717,Asia/Jakarta,6P58QR66+8G,False,4.4,4006,https://

In [26]:
where_condition=''
for google_id in tqdm(data.google_id.unique()):
    where_condition=where_condition+' '+f"""or lower(google_id) like lower('%{google_id}%')"""

delete_query=f"""
    DELETE FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_results`
    where{where_condition[3:]}
"""
query_job=cashbac_bqclient.query(delete_query).result(timeout=None)

delete_query=f"""
    DELETE FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_results_mall`
    where{where_condition[3:]}
"""
query_job=cashbac_bqclient.query(delete_query).result(timeout=None)




# delete_query=f"""
#     DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results`
#     where{where_condition[3:]}
# """
# query_job=horego_bqclient.query(delete_query).result(timeout=None)

# delete_query=f"""
#     DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results_mall`
#     where{where_condition[3:]}
# """
# query_job=horego_bqclient.query(delete_query).result(timeout=None)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2138/2138 [00:00<00:00, 130739.49it/s]


In [27]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_delete_in_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_delete_in_prod.csv',sep=';',index=False)

CPU times: user 35.3 ms, sys: 20.8 ms, total: 56 ms
Wall time: 692 ms


In [15]:
save_columns

['google_id', 'place_id']

In [28]:
%%time
cashbac_key='Cashbac-GCP-Keys.json'
# horego_key='Horego-GCP-Keys.json'

#############################################################################################

src_path_file='results/input_list_delete_in_prod.csv'
target_path_file='scrapping_results/input_list_delete_in_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
#table_schema = "cashbac_datalake_dev"

table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_delete_in_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_delete_in_prod.csv
table on : cashbac_datalake_prod.list_delete_in_prod
CPU times: user 183 ms, sys: 23.3 ms, total: 206 ms
Wall time: 2.39 s


In [12]:
import psycopg2.pool
def db_connect_cashbac_nopool(host,database,user,password):
    """
    :return:
    """
    connection = psycopg2.connect(host=host,
                              database=database,
                              user=user,
                              password=password,
                              port=5432)
    return connection

In [17]:
subtype_condition=''
for subtype in subtype_exclude.KEYWORD.unique():
    subtype_condition=subtype_condition+' '+f"""or lower(outlet_tags_en::varchar) like lower('%{subtype}%')"""

name_condition=''
for name in name_exclude.KEYWORD.unique():
    name_condition=name_condition+' '+f"""or lower(outlet_name) like lower('%{name}%')"""

exclude_restorant_condition=''
for place_id in exclude_restorant.place_id.unique():
    exclude_restorant_condition=exclude_restorant_condition+' '+f"""or lower(google_place_id) = lower('{place_id}')"""


include_restorant_condition='lower(google_place_id) not in ('
for place_id in include_restorant[(include_restorant.place_id!='')].place_id.unique():
    place_id=place_id.lower()
    include_restorant_condition=include_restorant_condition+' '+f"""'{place_id}',"""
include_restorant_condition=include_restorant_condition[:-1]+')'


include_restorant_condition_google_id='lower(google_id) not in ('
for google_id in include_restorant[(include_restorant.google_id!='')].google_id.unique():
    google_id=google_id.lower()
    include_restorant_condition_google_id=include_restorant_condition_google_id+' '+f"""'{place_id}',"""
include_restorant_condition_google_id=include_restorant_condition_google_id[:-1]+')'


include_restorant_condition2='lower(google_place_id) not in ('
for place_id in outlet_in_mall_type1[(outlet_in_mall_type1.place_id!='')].place_id.unique():
    place_id=place_id.lower()
    include_restorant_condition2=include_restorant_condition2+' '+f"""'{place_id}',"""
include_restorant_condition2=include_restorant_condition2[:-1]+')'


include_restorant_condition_google_id2='lower(google_id) not in ('
for google_id in outlet_in_mall_type1[(outlet_in_mall_type1.google_id!='')].google_id.unique():
    google_id=google_id.lower()
    include_restorant_condition_google_id2=include_restorant_condition_google_id2+' '+f"""'{google_id}',"""
include_restorant_condition_google_id2=include_restorant_condition_google_id2[:-1]+')'

In [16]:
%%time
from io import StringIO # python3; python2: BytesIO

query_string=f""" 

select * from 
(
    SELECT * 
    FROM 
    (
        select * from cbmerchant.outlet
    )table1
    where 
    (
        {subtype_condition[3:]}
    )
    or 
    (
        {name_condition[3:]}
    )
    or 
    (
        {exclude_restorant_condition[3:]}
    )
)table1
where true
and status = 'ACTIVE'
and 
(
    {include_restorant_condition}
)
and 
(
    {include_restorant_condition_google_id}
)
and 
(
    {include_restorant_condition2}
)
and 
(
    {include_restorant_condition_google_id2}
)
"""

# with db_connect_cashbac_nopool('uat-cashbac-db.ciwxizsaix0m.ap-southeast-1.rds.amazonaws.com','cashbacdb','data_ds','data123') as conn:
with db_connect_cashbac_nopool('cashbac-db-zcb.cc5hge1ycpej.ap-southeast-3.rds.amazonaws.com','cashbacdb','readonly','r34d0nly4321') as conn: 
    csv_buffer = StringIO()
    data =pd.read_sql_query(query_string, conn)

conn.close()

NameError: name 'db_connect_cashbac_nopool' is not defined

In [25]:
data=data[['outlet_pubid','google_id','google_place_id']]
data.columns=['outlet_pubid','google_id','place_id']

In [26]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_delete_in_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_delete_in_prod.csv',sep=';',index=False)

CPU times: user 23 ms, sys: 21.7 ms, total: 44.7 ms
Wall time: 859 ms


In [28]:
%%time
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

src_path_file='results/input_list_delete_in_prod.csv'
target_path_file='scrapping_results/input_list_delete_in_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_delete_in_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_delete_in_prod.csv
table on : cashbac_datalake_prod.list_delete_in_prod
CPU times: user 314 ms, sys: 45.6 ms, total: 360 ms
Wall time: 3.48 s


#### SET NON OPERATIONAL INTO DEACTIVE

In [43]:

query_string=f""" 

select * 
from
( 
  SELECT 
  outlet_pubid
  ,google_id
  ,google_place_id place_id
  ,status
  ,table2.business_status
  FROM `cashbac-31433.cashbac_data_cbmerchant_prod.outlet` table1
  ,`cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results` table2 
  where true 
  and table1.google_place_id=table2.place_id
)where status='ACTIVE'
and business_status = 'CLOSED_PERMANENTLY'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
# print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

In [44]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_delete_in_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_delete_in_prod.csv',sep=';',index=False)

CPU times: user 26.8 ms, sys: 36.9 ms, total: 63.7 ms
Wall time: 1.14 s


In [46]:
%%time
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

src_path_file='results/input_list_delete_in_prod.csv'
target_path_file='scrapping_results/input_list_delete_in_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_delete_in_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_delete_in_prod.csv
table on : cashbac_datalake_prod.list_delete_in_prod
CPU times: user 336 ms, sys: 35 ms, total: 371 ms
Wall time: 3.6 s


##### DELETE UNWANTED NAME

In [8]:
try:
    cursor.close()
    conn_details.close()
except:
    pass
    
conn_details=db_connect_cashbac_nopool('uat-cashbac-db.ciwxizsaix0m.ap-southeast-1.rds.amazonaws.com','cashbacdb','data_ds','data123')
cursor = conn_details.cursor()

for name in tqdm(name_exclude.KEYWORD):

    delete_query=f"""
        DELETE FROM cbmerchant.outlet
        WHERE lower(outlet_name) like lower('%{name}%')
    """

    cursor.execute(delete_query)
    conn_details.commit()
    
cursor.close()
conn_details.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:13<00:00,  2.79it/s]


In [13]:
# where_condition=''
# for name in tqdm(name_exclude.KEYWORD.unique()):
#     where_condition=where_condition+' '+f"""or lower(name) like lower('%{name}%')"""

# delete_query=f"""
#     DELETE FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results`
#     where{where_condition[3:]}
# """
# query_job=cashbac_bqclient.query(delete_query).result(timeout=None)

# delete_query=f"""
#     DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results`
#     where{where_condition[3:]}
# """
# query_job=horego_bqclient.query(delete_query).result(timeout=None)

In [12]:
name_exclude.KEYWORD[49]

"de'cafe (car wash- fl detailing- cafe - vape shop)"

In [14]:
# for name in tqdm(name_exclude.KEYWORD):
    
#     delete_query=f"""
#         DELETE FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results`
#         WHERE lower(name) like lower('%{name}%')
#     """
#     query_job=cashbac_bqclient.query(delete_query).result(timeout=None)
    
#     delete_query=f"""
#         DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results`
#         WHERE lower(name) like lower('%{name}%')
#     """
#     query_job=horego_bqclient.query(delete_query).result(timeout=None)

#### DELETE PLACES FROM LIST

In [15]:
import gspread as gs
gc = gs.service_account(filename=horego_key)
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1H9YtIjyPaW6hVSWbGh65KU0f9Q97QgoALp-nBsBvo3g/edit#gid=0')
worksheet = sheet.worksheet('Exclude Restaurant')
excluded_resto = pd.DataFrame(worksheet.get_all_values())
excluded_resto.columns=excluded_resto.iloc[0]
excluded_resto=excluded_resto[1:]
excluded_resto=excluded_resto[excluded_resto['Place Id']!='']

In [16]:
list_place_id=''
for place_id in tqdm(excluded_resto['Place Id']):
    list_place_id=list_place_id+"'"+str(place_id)+"',"
list_place_id=list_place_id[:-1]

# for place_id in tqdm(excluded_resto['Place Id']):
    
delete_query=f"""
    DELETE FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results`
    WHERE true
    and lower(place_id) in ({list_place_id})
"""
query_job=cashbac_bqclient.query(delete_query).result(timeout=None)

delete_query=f"""
    DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results`
    WHERE true
    and lower(place_id) in ({list_place_id})
"""
query_job=horego_bqclient.query(delete_query).result(timeout=None)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 369/369 [00:00<00:00, 481099.84it/s]


In [12]:
try:
    cursor.close()
    conn_details.close()
except:
    pass
    
conn_details=db_connect_cashbac_nopool('uat-cashbac-db.ciwxizsaix0m.ap-southeast-1.rds.amazonaws.com','cashbacdb','data_ds','data123')
cursor = conn_details.cursor()

for place_id in tqdm(excluded_resto['Place Id']):

    delete_query=f"""
        DELETE FROM cbmerchant.outlet
        WHERE lower(google_place_id) like lower('%{place_id}%')
    """

    cursor.execute(delete_query)
    conn_details.commit()
    
cursor.close()
conn_details.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 339/339 [02:28<00:00,  2.28it/s]


### REQUEST DELETE 

In [3]:
data_delete=pd.read_excel('results/Master Data Outlet.xlsx',sheet_name='Master')
data_delete=data_delete[data_delete['outlet_pubid'].isnull()==False]
data_delete.head()

,merchant_pubid,merchant_name,outlet_pubid,create_ts_outlet,outlet_name,phone_number,building_name,address,province,city,sub_district,urban_village,business_type,merchant_status,merchant_option,outlet_status,outlet_option,use_new_cashback_scheme,Commision (New User),Commision (Retain User),Commision (Lapsed User),Reg Cashback (New User),Reg Cashback (Retain User),Reg Cashback (Lapsed User),instrument,terminal_model,software_version,terminal_model_status,terminal_id,activated_date,beaconid,Type Of Cashbac,Region,Type Of Building,Type Of Merchant,Payment Interface,Software Manufature,Outlet Status (Update),Merchant Status (Update),Date,Merchant Type,URL,Year,Month,Week,Unnamed: 45
0,37021221-286f-412c-a57c-22c794e20797,BBQ CHICKEN & BEER,77825a18-d037-4c95-925c-aa2b2ec72414,2017-11-10 15:33:43,BBQ CHICKEN & BEER RUKAN CROWN GOLF,81311664481,Rukan Crown Golf,"Rukan Crown Golf, Bukit Golf Mediterania, Nomo...",DKI JAKARTA,KOTA JAKARTA UTARA,KAMAL MUARA,PENJARINGAN,FOOD,ACTIVE,"{CASHBAC_PAY,CASHBAC_DEALS}",TERMINATED,"{CASHBAC_PAY,CASHBAC_DEALS}",0.0,3.0,2.0,2.0,9.0,6.0,10.0,Beacon,POS-Quinos,1,INACTIVE,BBQCHICKEN1,2017-11-10 16:23:53,3.134307e+11,CASHBAC_PAY,JABODETABEK,Non Mall,NON CHAIN,Beacon,POS-Quinos,NaN,NaN,2017-11-10,Offline,NaN,2017,11,45,1
1,d1f7f6cd-c017-44fc-9659-de1b0423f691,BEBEK BENGIL,5febe3c3-3e24-41a8-93a6-456b2c65117d,2017-11-10 16:00:44,BEBEK BENGIL PLAZA INDONESIA,2129921882,Plaza Indonesia,"Plaza Indonesia, Lantai 2, Jalan MH. Thamrin, ...",DKI JAKARTA,KOTA JAKARTA PUSAT,GONDANGDIA,MENTENG,FOOD,ACTIVE,"{CASHBAC_PAY,CASHBAC_DEALS}",ACTIVE,{CASHBAC_PAY},0.0,5.5,5.5,5.5,4.0,4.0,4.0,Beacon,Offline-Cashbac,1,ACTIVE,1,2017-11-10 16:27:32,3.764634e+11,CASHBAC_PAY,JABODETABEK,Mall,NON CHAIN,Beacon,Offline-Cashbac,NaN,NaN,2017-11-10,Offline,NaN,2017,11,45,4
2,e3effdd7-751a-4d20-81bb-c7372c97e279,DOUBLE SPOUT COFFEE,722bd2cd-7b2e-4a7b-9153-1708ac1d3db1,2017-11-10 17:55:15,DOUBLE SPOUT COFFEE COHIVE D.LAB,87877170719,Cohive D.Lab,"CoHive D.Lab, Lantai 2, Jalan Riau Nomor 1, Go...",DKI JAKARTA,KOTA JAKARTA PUSAT,GONDANGDIA,MENTENG,FOOD,ACTIVE,"{CASHBAC_PAY,CASHBAC_DEALS}",TEMPORARILY CLOSED,"{CASHBAC_PAY,CASHBAC_DEALS}",0.0,2.0,1.0,2.0,8.0,4.0,8.0,Beacon,App-Cashbac,1,INACTIVE,DOUBLESPOUT1,2017-11-10 18:04:05,6.231316e+11,CASHBAC_PAY,JABODETABEK,Non Mall,NON CHAIN,Beacon,App-Cashbac,NaN,NaN,2017-11-10,Offline,NaN,2017,11,45,1
3,77030f48-12d3-40fc-a357-207c862c3259,LATTICE CAFE,7c05d605-626a-4a15-b9b7-4c0e3462c067,2017-11-13 14:07:20,LATTICE CAFE SINARMAS LAND PLAZA,NaN,Sinarmas Land Plaza,"Sinarmas Land Plaza, Lantai G, Jalan MH. Thamr...",DKI JAKARTA,KOTA JAKARTA PUSAT,GONDANGDIA,MENTENG,FOOD,DEACTIVATED,"{CASHBAC_PAY,CASHBAC_DEALS}",TERMINATED,{CASHBAC_PAY},0.0,2.0,2.0,2.0,8.0,6.0,8.0,Beacon,App-Cashbac,1,INACTIVE,LATTICE1,2017-11-13 14:16:49,3.637653e+11,CASHBAC_PAY,JABODETABEK,Non Mall,NON CHAIN,Beacon,App-Cashbac,NaN,NaN,2017-11-13,Offline,NaN,2017,11,46,1
4,194753ca-3536-4861-8096-1c28e395ebe2,TAAGAR,300783f2-269d-446e-8578-e8aa161ab5ea,2017-11-15 13:57:24,TAAGAR MAMPANG PRAPATAN,87783386337,NaN,"Jalan Kemang Raya Nomor 130 D, Bangka, Mampang...",DKI JAKARTA,KOTA JAKARTA SELATAN,BANGKA,MAMPANG PRAPATAN,FOOD,ACTIVE,"{CASHBAC_PAY,CASHBAC_DEALS}",TERMINATED,"{CASHBAC_PAY,CASHBAC_DEALS}",0.0,3.0,3.0,3.0,9.0,9.0,9.0,Beacon,App-Cashbac,1,INACTIVE,KEDAITAAKIS1,2017-11-16 17:40:49,6.561657e+11,CASHBAC_PAY,JABODETABEK,Non Mall,NON CHAIN,Beacon,App-Cashbac,NaN,NaN,2017-11-15,Offline,NaN,2017,11,46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22159,f7201829-0645-4934-bed1-84f68b20df72,FAMILYMART,a20bdbbe-e2ac-4af8-9deb-5934afc9e906,2023-06-14 14:18:02,FAMILYMART PALMERAH,81119278287,NaN,"Jalan Palmerah Timur Nomor 13, Gelora, Tanah A...",DKI JAKARTA,KOTA JAKARTA PUSAT,GELORA,TANAH ABANG,GROCERY,ACTIVE,"{CASHBAC_PAY,CASHBAC_DEALS}",INITIATED,{CASHBAC_PAY},0.0,3.4,3.4,3.4,1.0,1.0,1.0,Q

In [67]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_delete_in_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data_delete.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data_delete[save_columns].drop_duplicates().to_csv('results/input_list_delete_in_prod.csv',sep=';',index=False)

CPU times: user 67.4 ms, sys: 81.4 ms, total: 149 ms
Wall time: 1.66 s


In [68]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

src_path_file='results/input_list_delete_in_prod.csv'
target_path_file='scrapping_results/input_list_delete_in_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_dev"
table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

table_schema = "cashbac_datalake_prod"
table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_delete_in_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_delete_in_prod.csv
table on : cashbac_datalake_dev.list_delete_in_prod
table on : cashbac_datalake_prod.list_delete_in_prod


In [5]:
query="""delete from cbmerchant.outlet where outlet_pubid in ("""
for outlet_pubid in data_delete['outlet_pubid']:
    # print(outlet_pubid)
    query=query+f"'{outlet_pubid}',"
query=query[:-1]+')'


In [6]:
query

"delete from cbmerchant.outlet where outlet_pubid in ('77825a18-d037-4c95-925c-aa2b2ec72414','5febe3c3-3e24-41a8-93a6-456b2c65117d','722bd2cd-7b2e-4a7b-9153-1708ac1d3db1','7c05d605-626a-4a15-b9b7-4c0e3462c067','300783f2-269d-446e-8578-e8aa161ab5ea','55eb369b-8acc-4eb3-a354-531b0b33cf77','db2c6b44-1cb5-47e7-9d6f-2656d028505c','68def11f-1616-4408-86f6-681ad3a76872','c5dc359f-0971-4c88-9807-06c3deaadc6b','57aa2310-1551-4751-be41-d916a1b4b464','577efd02-ecf4-460c-acdf-b059a9f6c286','60ef3698-c3d6-42e7-9e6e-5b92f10c550b','3818ce07-88dc-4efe-98f2-00e615740dc9','01eea6aa-ed96-4247-b4a2-2ef9a12570ae','9ff0de73-138b-4fa4-b2d1-9f68989be5ff','3af7f00d-ebbf-427c-a078-6d503f6021ec','e6968639-6173-4bde-ac19-5b0624647952','a368659c-857c-4585-9257-37516a7e67d9','e4183db1-44b6-4d4f-8a33-04d73f8b06f9','e6f63596-778f-4c8a-8420-af35ce08fe38','7905fff1-bfdf-42ac-a283-0a2256ec7f81','654967c6-2c89-4d81-9927-9536f30ebb49','c2d97836-59d6-43ce-a0c0-118160809062','c37e462a-a30d-4759-b780-8af9ee0849a5','073f8af7-

In [80]:
len(data_delete['outlet_pubid'])

22164

In [7]:
file = open("sample.txt", "w")

query="""
update cbmerchant.outlet 
set status = 'DEACTIVATED'
where outlet_pubid in ("""

file.write(query + "\n")
for outlet_pubid in data_delete['outlet_pubid']:
    file.write("'"+outlet_pubid + "', \n")
    # query=query+f"'{outlet_pubid}',"
file.write(")")
file.close()